In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import time
import os
import glob
from collections import deque

In [2]:
# READ-IN CLEAN ARC DATAFRAME
filename = 'data/MEBL3/MEBL3_All_Arc_Counts_Clean'
arc_counts_df = pd.read_csv(filename, parse_dates=['Time']).fillna(0) # 
print(f"{filename} : file read into a pandas dataframe.")
#arc_count_df["Time"] = arc_count_df["Time"].astype(str)
arc_counts_df

data/MEBL3/MEBL3_All_Arc_Counts_Clean : file read into a pandas dataframe.


,Time,B2 Beam Blanker,B2 Bias,B2 Extractor,B2 Filament,B2 Gun Lens,B2 Main Lens,B2 Suppressor,B3 Beam Blanker,B3 Bias,...,D3 Gun Lens,D3 Main Lens,D3 Suppressor,D4 Beam Blanker,D4 Bias,D4 Extractor,D4 Filament,D4 Gun Lens,D4 Main Lens,D4 Suppressor
0,2024-06-14 01:55:12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-06-14 01:55:13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-06-14 01:55:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-06-14 01:55:15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-06-14 01:55:16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060403,2024-06-27 23:59:54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1060404,2024-06-27 23:59:55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1060405,2024-06-27 23:59:56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1060406,2024-06-27 23:59:57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# CONVERT ARC DATA FROM PANDAS DATAFRAME TO NUMPY ARRAY (NP RUNS INTERNALLY ON C CODE, THUS FASTER TO WORK WITH ON LARGE DATASETS)
arc_counts_arr = arc_counts_df.to_numpy()
arc_counts_arr

array([[Timestamp('2024-06-14 01:55:12'), 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [Timestamp('2024-06-14 01:55:13'), 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [Timestamp('2024-06-14 01:55:14'), 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [Timestamp('2024-06-27 23:59:56'), 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [Timestamp('2024-06-27 23:59:57'), 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [Timestamp('2024-06-27 23:59:58'), 0.0, 0.0, ..., 0.0, 0.0, 0.0]],
      dtype=object)

In [4]:
# READ IN PRESSURE DATA AS PANDAS DATAFRAME
filename = 'data/MEBL3/MEBL3_Pressure-data-2024-06-28 09_40_08.csv'
pressure_df = pd.read_csv(filename, parse_dates=['Time']).fillna(0)    #
print(f"{filename} : file read into a pandas dataframe.")
#pressure_df["Time"] = pressure_df["Time"].astype(str)
pressure_df["Chamber Pressure"] = pressure_df["Chamber Pressure"].astype(np.float64)
pressure_df["Column Pressure"] = pressure_df["Column Pressure"].astype(np.float64)
pressure_df

data/MEBL3/MEBL3_Pressure-data-2024-06-28 09_40_08.csv : file read into a pandas dataframe.


,Time,Chamber Pressure,Column Pressure
0,2024-06-14 01:55:29,1.850000e-07,5.550000e-10
1,2024-06-14 01:55:30,1.840000e-07,5.670000e-10
2,2024-06-14 01:55:31,1.830000e-07,5.540000e-10
3,2024-06-14 01:55:32,1.800000e-07,5.610000e-10
4,2024-06-14 01:55:33,1.810000e-07,5.490000e-10
...,...,...,...
1059524,2024-06-27 23:59:54,1.650000e-07,3.380000e-10
1059525,2024-06-27 23:59:55,1.630000e-07,3.370000e-10
1059526,2024-06-27 23:59:56,1.640000e-07,3.410000e-10
1059527,2024-06-27 23:59:57,1.650000e-07,3.380000e-10


In [5]:
# CONVERT PRESSURE DATA TO NUMPY ARRAY
pressure_arr = pressure_df.to_numpy()
pressure_arr

array([[Timestamp('2024-06-14 01:55:29'), 1.85e-07, 5.55e-10],
       [Timestamp('2024-06-14 01:55:30'), 1.84e-07, 5.67e-10],
       [Timestamp('2024-06-14 01:55:31'), 1.83e-07, 5.54e-10],
       ...,
       [Timestamp('2024-06-27 23:59:56'), 1.64e-07, 3.41e-10],
       [Timestamp('2024-06-27 23:59:57'), 1.65e-07, 3.38e-10],
       [Timestamp('2024-06-27 23:59:58'), 1.64e-07, 3.4e-10]],
      dtype=object)

In [6]:
# GRAB TIMES OF ALL ARC COUNTS (FOR EACH COMPONENT)
def get_arc_times(col_hvps):
    '''Given a column HVSP, find Time values of all arc events
        Argument
        col_idx:   type=int            index of the column HVSP of measure within arc_counts
        Return
        arc_times_df: type=np.array    numpy array arc_counts filtered to keep just arc events of column HVSP of measure
    '''
    #arc_times = arc_counts_df[arc_counts_df[col_hvps] > 0]["Time"]
    arc_times = arc_counts_df[arc_counts_df[col_hvps] > 0]["Time"]
    if len(arc_times) > 0: 
        arc_idx = np.where(pressure_df["Time"].isin(arc_times))[0]
        #print(f"{arc_times=}")
        #print()
        #print(arc_times.iloc[0])
        #print()
        #print(f"{arc_idx=}")
    else: arc_idx = []
    return arc_times, arc_idx

In [7]:
# FILTER PRESSURE DATA
def pressure_window(col_hvps, arc_times, arc_time_idx, time_range=10):
    pressure = []
    #all_times = pressure_df["Time"]
    num_rows = pressure_df.shape[0]
    for arc, idx in enumerate(arc_time_idx):
        #print(i)
        #print(f"{row_idx=}, {num_rows=}")
        if idx < time_range:
            pressure = pressure_arr[:idx+time_range+1]
        elif idx + time_range > num_rows:
            pressure = pressure_arr[idx-time_range:]
        else:
            pressure = pressure_arr[idx-time_range:idx+time_range+1]
        if pressure.size > 0:
            local_chamber_mean = np.mean(pressure[:,1])
            local_chamber_max = np.max(pressure[:,1])
            local_column_mean = np.mean(pressure[:,2])
            local_column_max = np.max(pressure[:,2])
            #print(pressure)
            #print(pressure.shape)
            #print(pressure.dtype)
            #pressure_plot = pd.DataFrame(pressure, columns=["Time", "Chamber Pressure", "Column Pressure"], dtype={"Time": "string", "Chamber Pressure": "float", "Column Pressure": "float"})
            #print(pressure_plot)
            pressure_at_arc_plot(col_hvps, arc_times.iloc[arc], pressure, local_chamber_mean, local_chamber_max, local_column_mean, local_column_max)
    #if len(pressure_window_arr) > 0:
    #    print(f"{arc_times=}")
    #    print(f"{[x[0] for x in pressure_window_arr]=}")
    return pressure
# should i create new measurements of variance from mean?

In [22]:
def pressure_at_arc_plot(col_hvps, arc, pressure, ch_mean, ch_max, co_mean, co_max):
    # INITIALIZE
    plt.figure(figsize=(12, 6))
    ax1 = plt.gca()
    # PLOT VARIABLES 
    time = pressure[:,0]
    cham_pres = pressure[:, 1]
    col_pres = pressure[:, 2]
    # CHAMBER PRESSURE
    sns.lineplot(x=time, y=cham_pres, ax=ax1, label='Chamber Pressure', color='b')     # chamber pressure
    ax1.set_ylabel('Chamber Pressure')
    ax1.set_xlabel('Time')
    ax1.axhline(y=ch_mean, color='b', linestyle='--', linewidth=0.75, label='Chamber Pressure Mean')
    ax1.legend(loc='upper left')
    # COLUMN PRESSURE
    ax2 = ax1.twinx()
    sns.lineplot(x=time, y=col_pres, ax=ax2, label='Column Pressure', color='r')
    ax2.set_ylabel('Column Pressure')
    ax2.set_xlabel('Time')
    ax2.axhline(y=co_mean, color='r', linestyle='--', linewidth=0.75, label='Column Pressure Mean')
    ax2.legend(loc='upper right')
    ax1.axvline(x=arc, color='black', linewidth=0.75)
    #ax1.axhline(y=ch_max)
    #ax2.axhline(y=co_max)
    # CLEAN
    title = f'System Pressure at {arc} Arc Event on {col_hvps}'
    ax1.set_title(title)
    newpath = f'plots/pressure_at_arcs/{col_hvps}'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    plt.savefig(f"{newpath}/{((str(arc).replace(" ", "__")).replace(":", "_")).replace("-", "_")}")
    plt.show()

In [ ]:
# MAIN LOOP
start_time = time.time()
pd.options.display.float_format = '{:.2e}'.format
col_hvps_list = list(arc_counts_df.columns[1:])                                      # list of all Column HVPS to be looped through
times = arc_counts_df["Time"]                                                            # get all Time values
for col_idx, col_hvps in enumerate(col_hvps_list):
    # TERMINAL OUTPUT READABILITY
    lenght_symbol = (74-len(col_hvps))//2
    print(f"{'+'*lenght_symbol}{col_hvps}{'+'*lenght_symbol}")
    if col_hvps == 'C3 Suppressor':
        continue
    # GET ARC TIMES
    arc_times, arc_time_idx = get_arc_times(col_hvps)                                        # col_idx from enumerate will be offset by 1 from proper index in arc_counts
    # FILTER PRESSURE DATA BASED ON ARC TIMES
    pressure = pressure_window(col_hvps, arc_times, arc_time_idx)
    ## save data??

    #if len(pressure) > 0:
    #    break
end_time = time.time()
elapsed_time = end_time-start_time
print(f"{elapsed_time=}")

# Baseline: 55.4 seconds
# Current best: 50.4 seconds, 9.9% faster